In [13]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin

# pts1 = gpd.read_file('../targets_0_sar/ITA/sar_ITA_e_v2.geojson')
# pts2 = gpd.read_file('../targets_0_sar/ITA/sar_ITA_w_v2.geojson')
pts = gpd.read_file('../targets_0_sar/ESP/sar_ESP_v2.geojson')
#pts = pd.concat([pts1, pts2])
#print(len(pts1))
#print(len(pts2))
print(len(pts))

2648


In [14]:
#load images
#n = 'italy'
n = 'spain'
folder = '/Users/Zack/0_thesis_bing/imagery_' + n + '/'
files = os.listdir(folder)
print(len(files))

#extract coordinates
x_list, y_list = [], []
for i in range(len(files)):
    #extract bottom right coordinate from image name
    img = files[i]
    x1 = float(img.split(',')[1])
    y1 = float(img.split(',')[2])
    x2 = float(img.split(',')[3])
    y2 = float((img.split(',')[4]).split(')')[0])
    x = (x1+x2)/2
    y = (y1+y2)/2
    x_list.append(x)
    y_list.append(y)

1182


In [15]:
geometry = [Point(i) for i in zip(y_list, x_list)]
missing = gpd.GeoDataFrame(geometry=geometry, crs={'init' :'EPSG:4326'})
len(missing)

1182

In [16]:
#missing = missing.to_crs({'init': 'EPSG:7794'}) #ITA
missing = missing.to_crs({'init': 'EPSG:2062'}) #ESP
mask = gpd.GeoDataFrame(geometry = missing.buffer(10))
mask['Dissolve'] = 0
mask = mask.dissolve('Dissolve')
mask = mask.to_crs({'init': 'EPSG:4326'})

In [17]:
clip = sjoin(pts, mask, how='inner', op='within')
print(len(clip))
clip.head()

1182


,geometry,index_right
0,POINT (-1.391646398701104 36.44488163528315),0
1,POINT (-1.766479749304226 36.36067806002004),0
2,POINT (-2.24967854680042 36.34506837156938),0
3,POINT (-2.749406409434271 36.33775408387967),0
4,POINT (-0.32758660941558 36.91924779218665),0


In [18]:
clip.to_file('../missing_' + n + '.geojson', driver='GeoJSON')
clip.to_file('../missing_' + n + '.shp')
#mask.to_file('../missing_' + n + '_buff.geojson', driver='GeoJSON')